In [13]:
import numpy as np
import pandas as pd

from loguru import logger

In [23]:
def read_csv(path: str) -> pd.DataFrame:
    file = pd.read_csv(path)
    return file

TRAIN_SET = read_csv("./train.csv")
TEST_SET = read_csv("./test.csv")
TRAIN_SET.head()

,id,text,emotions
0,27383,i feel awful about it too because it s my job ...,sadness
1,110083,im alone i feel awful,sadness
2,140764,ive probably mentioned this before but i reall...,joy
3,100071,i was feeling a little low few days back,sadness
4,2837,i beleive that i am much more sensitive to oth...,love


In [4]:
dataset = TRAIN_SET
dataset.emotions.value_counts()

joy         412
sadness     351
anger       161
fear        129
love        106
surprise     41
Name: emotions, dtype: int64

In [10]:
dataset_js = [
    {
        "id": 27383,
        "text": "i feel awful about it too because it s my job to get him in a position to succeed and it just didn t happen here",
        "emotion": "sadness"
    },
    {
        "id": 110083,
        "text": "im alone i feel awful	",
        "emotion": "sadness"
    }
]
# 
dataset = dataset_js
texts = [entry['text'] for entry in dataset_js]
emotions = [entry['emotion'] for entry in dataset_js]
texts, emotions

(['i feel awful about it too because it s my job to get him in a position to succeed and it just didn t happen here',
  'im alone i feel awful\t'],
 ['sadness', 'sadness'])

In [13]:
emotion_set = set(emotions)
emotion_to_int = {emotion: i for i, emotion in enumerate(emotion_set)}
int_to_emotion = {i: emotion for i, emotion in enumerate(emotion_set)}
emotion_to_int, int_to_emotion

({'sadness': 0}, {0: 'sadness'})

In [7]:
import nltk
from nltk.corpus import brown
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [32]:
word2vec = {}
word_set = {}
word_all = []
idx = 0
for dataset in (TRAIN_SET, TEST_SET):
    for text in dataset.text:
        words = text.split()
        for word in words:
            if word not in word2vec:
                word_all.append(word)
                word2vec[word] = idx
                idx += 1
    word_set = set(word_all)
    # logger.debug(idx)
logger.info(len(word2vec))
# logger.info(len(word_set))
    # print(text)
    # break

2023-02-13 14:53:00.609 | INFO     | __main__:<module>:15 - 4895


In [60]:
text_features_train = []
for text in TRAIN_SET.text:
    features = [0] * len(word2vec)
    words = text.split()
    for word in words:
        features[word2vec[word]] = 1
    text_features_train.append(features)
logger.info(len(text_features_train[0]))


2023-02-13 15:09:05.257 | INFO     | __main__:<module>:8 - 4895
2023-02-13 15:09:05.259 | INFO     | __main__:<module>:9 - 1200


In [61]:
text_features_test = []
for text in TEST_SET.text:
    features = [0] * len(word2vec)
    words = text.split()
    for word in words:
        features[word2vec[word]] = 1
    text_features_test.append(features)
logger.info(len(text_features_test[0]))
logger.info(len(text_features_test))

2023-02-13 15:09:05.904 | INFO     | __main__:<module>:8 - 4895
2023-02-13 15:09:05.905 | INFO     | __main__:<module>:9 - 800


In [54]:
emotions = [_ for _ in TRAIN_SET.emotions]
# emotions_test = [_ for _ in TEST_SET.emotions]

# emotion_set = set(emotions)
emotion_set = set(emotions)

emotion_to_int = {emotion: i for i, emotion in enumerate(emotion_set)}
int_to_emotion = {i: emotion for i, emotion in enumerate(emotion_set)}

In [55]:
emotions = [emotion_to_int[emotion] for emotion in emotions]
train_targets = np.zeros((len(emotions), len(emotion_set)))
for i, emotion in enumerate(emotions):
    train_targets[i, emotion] = 1

In [65]:
np.array(text_features_train).shape

(1200, 4895)

In [72]:
weights = np.zeros((len(word2vec), len(emotion_set)))
learning_rate = 0.01
num_epochs = 100
for epoch in range(num_epochs):
    scores = np.dot(np.array(text_features_train), weights)
    predictions = 1 / (1 + np.exp(-scores))
    # logger.debug(np.array(predictions).shape)
    # logger.debug(np.array(train_targets).shape)
    error = train_targets - predictions
    gradient = np.dot(np.array(text_features_train).T, error)
    weights += learning_rate * gradient
    logger.info(f"Epoch = {epoch}")

# Predict emotions for the test data
scores = np.dot(text_features_test, weights)
predictions = 1 / (1 + np.exp(-scores))
predictions = np.round(predictions)
predicted_emotion = int_to_emotion[np.argmax(predictions[0])]

2023-02-13 15:12:26.809 | INFO     | __main__:<module>:12 - Epoch = 0
2023-02-13 15:12:27.490 | INFO     | __main__:<module>:12 - Epoch = 1
2023-02-13 15:12:28.124 | INFO     | __main__:<module>:12 - Epoch = 2
2023-02-13 15:12:28.764 | INFO     | __main__:<module>:12 - Epoch = 3
2023-02-13 15:12:29.396 | INFO     | __main__:<module>:12 - Epoch = 4
2023-02-13 15:12:30.028 | INFO     | __main__:<module>:12 - Epoch = 5
2023-02-13 15:12:30.669 | INFO     | __main__:<module>:12 - Epoch = 6
2023-02-13 15:12:31.312 | INFO     | __main__:<module>:12 - Epoch = 7
2023-02-13 15:12:31.951 | INFO     | __main__:<module>:12 - Epoch = 8
2023-02-13 15:12:32.584 | INFO     | __main__:<module>:12 - Epoch = 9
2023-02-13 15:12:33.215 | INFO     | __main__:<module>:12 - Epoch = 10
2023-02-13 15:12:33.844 | INFO     | __main__:<module>:12 - Epoch = 11
2023-02-13 15:12:34.460 | INFO     | __main__:<module>:12 - Epoch = 12
2023-02-13 15:12:35.123 | INFO     | __main__:<module>:12 - Epoch = 13
2023-02-13 15:12

In [119]:
pred_emotion = []

scores = np.dot(text_features_test, weights)
predictions = 1 / (1 + np.exp(-scores))
# logger.debug(predictions)
# predictions = np.ones(predictions.shape) * np.argmax(predictions, axis=1)
# len(np.argmax(predictions, axis=1))
pred = np.zeros(predictions.shape)
for idx in range(pred.shape[0]):
    # logger.debug(np.argmax(predictions, axis=1)[idx])
    pred[idx][np.argmax(predictions, axis=1)[idx]] = 1

for idx in range(pred.shape[0]):
    # logger.debug(np.where(pred[idx]==1))
    
    # logger.debug(int_to_emotion[np.where(pred[idx]==1)[0][0]])
    pred_emotion.append(int_to_emotion[np.where(pred[idx]==1)[0][0]])
# predictions = np.argmax(predictions)
# predicted_emotion = int_to_emotion[np.argmax(predictions[0])]


In [125]:
dataset['emotions'] = pred_emotion
dataset.to_csv("./test_lg.csv")

In [124]:
dataset

,id,text,emotions
0,139312,ive been feeling more optimistic this week tha...,sadness
1,110362,i suppose i was feeling adventurous and volunt...,joy
2,45691,i feel like when a cycle comes i get all depre...,sadness
3,958,when confronted and in my opinion hassled by t...,anger
4,33882,i seldom feel shaky mid run,fear
...,...,...,...
795,566,that was what i felt when i was finally accept...,joy
796,36236,i take every day as it comes i m just focussin...,sadness
797,76229,i just suddenly feel that everything was fake,sadness
798,131640,im feeling more eager than ever to claw back w...,joy
